In [2]:
pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 97.8 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import os

# Cargar imagen y convertir a gris
img_path = 'imagen.jpg'
img_color = cv2.imread(img_path)
img_gray  = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)

# Paso 1: Umbral fijo
_, th_fixed = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY_INV)

# Paso 2: Umbral adaptativo
th_adapt = cv2.adaptiveThreshold(
    img_gray, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV,
    blockSize=15,
    C=5
)

# Paso 3: Máscara combinada
mask = cv2.bitwise_or(th_fixed, th_adapt)

# Paso 4: Contornos
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
h, w = img_gray.shape
area_min = h*w * 0.002
contours = [c for c in contours if cv2.contourArea(c) > area_min]

# Paso 5: Dibujo final
out = img_color.copy()
for cnt in contours:
    x,y,ww,hh = cv2.boundingRect(cnt)
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00']) if M['m00'] else 0
    cy = int(M['m01']/M['m00']) if M['m00'] else 0
    cv2.drawContours(out, [cnt], -1, (0,255,0), 2)
    cv2.circle(out, (cx,cy), 5, (255,0,0), -1)
    cv2.rectangle(out, (x,y), (x+ww, y+hh), (0,0,255), 2)

# Guardar frames temporales
frames = []
steps = [
    ("Original", img_color),
    ("Grises", cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)),
    ("Umbral fijo", cv2.cvtColor(th_fixed, cv2.COLOR_GRAY2BGR)),
    ("Umbral adaptativo", cv2.cvtColor(th_adapt, cv2.COLOR_GRAY2BGR)),
    ("Máscara combinada", cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)),
    ("Resultado final", out)
]

for i, (title, img) in enumerate(steps):
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax.set_title(title)
    ax.axis('off')
    fname = f"frame_{i}.png"
    fig.savefig(fname)
    frames.append(imageio.imread(fname))
    plt.close()
    os.remove(fname)  # elimina archivos temporales

# Guardar el GIF
imageio.mimsave('resultado.gif', frames, duration=1.2)
